# SageMaker Endpoint 생성 - KoSimCSE-RoBERTa를 사용한 한국어 문장 임베딩
이 워크샵에서는 Hugging Face의 KoSimCSE-RoBERTa 모델을 사용하여 한국어 문장 임베딩을 수행합니다. KoSimCSE-RoBERTa는 한국어 문장 임베딩에 특화된 모델로, 높은 수준의 의미론적 텍스트 유사성을 제공합니다.

## 주요 특징:
* 성능: KoSimCSE-RoBERTa는 다양한 텍스트 유사성 테스트 세트에서 뛰어난 성능을 보입니다. 다양한 유사성 지표에서 83% 이상의 높은 점수를 기록했습니다.[1]
* 사용 용이성: PyTorch와 transformers 라이브러리를 사용하여 쉽게 문장 임베딩을 생성할 수 있습니다.
* 활용 : 이 모델은 한국어 FAQ 문장셋의 임베딩을 수행하고, 생성한 문장 임베딩 벡터를 Faiss 엔진을 사용한 OpenSearch를 통해 검색하는 작업을 진행하는데 사용합니다.


Kernel
- conda_pytorch_p310 (SageMaker notebook 기준)
- Data Science (SageMaker studio 기준)

### Model Ref:
- [1]. BM-K/KoSimCSE-roberta
    - https://huggingface.co/BM-K/KoSimCSE-roberta
Inference Code Ref:    
- Huggingface Sagemaker-sdk - Deploy 🤗 Transformers for inference
    - https://github.com/huggingface/notebooks/blob/main/sagemaker/11_deploy_model_from_hf_hub/deploy_transformer_model_from_hf_hub.ipynb
- Sentence Embeddings with Hugging Face Transformers, Sentence Transformers and Amazon SageMaker - Custom Inference for creating document embeddings with Hugging Face's Transformers
    - https://github.com/huggingface/notebooks/blob/main/sagemaker/17_custom_inference_script/sagemaker-notebook.ipynb

## 0. Install packages

In [ ]:
install_needed = False  # should only be True once

In [ ]:
import sys
import IPython
import time

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U torch
    !{sys.executable} -m pip install -U transformers
    
    IPython.Application.instance().kernel.do_shutdown(True)

## 1. 로컬 모델 테스트: Huggingface로부터 모델과 토크나이저 로딩
* 로컬 환경에서 유사도 계산과 테스트를 위해 모델과 토크나이저 로드하기





In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

In [ ]:
model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta')
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta')

## 2. local test - sentence to embedding

In [ ]:
sample = "타기관OTP 이용등록방법 알려주세요"

In [ ]:
# 텐서플로우를 사용하여 토큰화된 샘플 데이터를 입력받고, 그 데이터를 임베딩하고 모델을 통해 출력하는 코드
#tokenizer() 함수는 토큰화된 데이터를 입력받고 padding과 truncation 옵션을 사용하여 데이터를 전처리. 그리고 return_tensors 옵션을 사용하여 데이터를 텐서로 반환
#모델을 통해 입력받은 데이터를 임베딩하고 return_dict 옵션을 사용하여 모델의 출력을 반환합니다.
inputs = tokenizer(sample, padding=True, truncation=True, return_tensors="pt")
embeddings, _ = model(**inputs, return_dict=False)


In [ ]:
emb_len = len(embeddings[0][0])
print("Sample Sentence: \n", sample)
print("Size of the Embedding Vector: ", emb_len)
print(f"First 10 Elements of the Embedding Vector (Total Elements: {emb_len}): \n", embeddings[0][0][0:10])

### 2.2 similarity
- 아래 첫문장, 두번째 문장의 유사도를 구함
- 아래 첫문장, 세째 문장의 유사도를 구함
- 최종적으로 유사도 수치를 비교 함

In [ ]:
#두 문장의 임베딩 점수를 계산하기 위한 함수
#토크나이저를 사용하여 문장을 토큰화하고 모델을 사용하여 임베딩을 계산한 다음, cal_score 함수를 사용하여 두 문장 사이의 임베딩 점수를 계산. 
#출력 결과는 두 문장 사이의 임베딩 점수
# 첫 기준문장과 두번째, 세번째 문장과의 유사도 
def show_embedding_score(tokenizer, model, sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    embeddings, _ = model(**inputs, return_dict=False)

    score01 = cal_score(embeddings[0][0], embeddings[1][0])
    score02 = cal_score(embeddings[0][0], embeddings[2][0])

    print(score01, score02)

In [ ]:
#이 함수는 두 개의 벡터 a, b를 입력받아 코사인 유사도를 구하는 함수입니다.
#벡터 a, b의 차원이 1이라면 unsqueeze 함수를 통해 차원을 늘려줍니다.
#그 다음 벡터 a, b를 각각 정규화하고, 두 벡터의 내적을 구하여 코사인 유사도를 구함. 
def cal_score(a, b):
    '''
    코사인 유사도 구하는 함수
    '''
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

In [ ]:
sentences1 = [sample,
             "타기관OTP 등록방법 알려줘요",
             '안녕 친구들!']

show_embedding_score(tokenizer, model, sentences1)    

## 3. SageMaker Endpoint에 embedding model 배포
* 로컬에서의 임베딩 테스트가 성공적으로 완료된 후, 다음 단계는 Amazon SageMaker를 사용하여 임베딩 모델을 실제 프로덕션 환경에 배포하는 것입니다. 
* SageMaker Endpoint를 생성함으로써, 어플리케이션 또는 서비스에서 API 호출을 통해 실시간으로 임베딩을 생성하거나 유사도를 계산할 수 있게 됩니다.
* 이 과정은 모델의 확장성과 가용성을 높이며, 더 큰 데이터셋에 대한 빠른 응답 시간을 보장합니다. SageMaker는 자동으로 모델을 로드 및 서빙하고, 필요에 따라 자동 확장을 수행할 수 있습니다.
* 이 단계를 완료하면, 임베딩 모델은 실제 서비스에서 사용할 준비가 되게 됩니다.

In [ ]:
import boto3
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

In [ ]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

print(f"sagemaker role arn: {role}")

### Endpoint 생성 시 허깅페이스 모델을 직접 로드하는 방식을 사용

In [ ]:
import threading
from sagemaker.huggingface import HuggingFaceModel

# Hub Model configuration
hub = {
  'HF_MODEL_ID': 'BM-K/KoSimCSE-roberta',
  'HF_TASK': 'feature-extraction'
}

# Create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,
   role=role,
   transformers_version="4.26",
   pytorch_version="1.13",
   py_version="py39",
)


In [ ]:
# from datetime import datetime
from sagemaker.utils import name_from_base

# Generate a unique endpoint name
# time_stamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
# endpoint_name = f"KoSimCSE-roberta-{time_stamp}"
endpoint_name = name_from_base("KoSimCSE-roberta")

# Function to deploy the model
def deploy_model():
    predictor = huggingface_model.deploy(
       initial_instance_count=1,
       endpoint_name=endpoint_name,
       instance_type="ml.g5.2xlarge"
    )
    print(f"Endpoint created: {endpoint_name}")

# Create a thread to run the deploy function in the background
deploy_thread = threading.Thread(target=deploy_model)

# Start the thread
deploy_thread.start()

# Optional: If you want to wait for the thread to complete
# deploy_thread.join()

print("Deployment is in progress in the background...")

In [ ]:
import time
from IPython.display import display, HTML
def make_console_link(region, endpoint_name, task='[SageMaker LLM Serving]'):
    endpoint_link = f'<b> {task} <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/endpoints/{endpoint_name}">Check Endpoint Status</a></b>'   
    return endpoint_link

def describe_endpoint(endpoint_name):
    '''
    엔드폰인트 생성 유무를 확인. 생성 중이면 기다림.
    '''
    sm_client = boto3.client("sagemaker")

    while(True):
        response = sm_client.describe_endpoint(
            EndpointName= endpoint_name
        )    
        status = response['EndpointStatus']
        if status == 'Creating':
            print("Endpoint is ", status)
            time.sleep(60)
        else:
            print("Endpoint is ", status)
            break

sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment

endpoint_link = make_console_link(region, endpoint_name)
display(HTML(endpoint_link))

In [ ]:
%%time
describe_endpoint(endpoint_name)         

# 4. Sagemaker Embedding Model Endpoint 추론 테스트

## Boto3 invoke_endpoint() 사용하여 추론

In [ ]:
import json
import boto3
import numpy as np
from sagemaker.predictor import Predictor

In [ ]:
predictor = Predictor(endpoint_name=endpoint_name)
predictor

In [ ]:
def query_endpoint_embedding_with_json_payload(encoded_json, endpoint_name, content_type="application/json"):
    client = boto3.client("runtime.sagemaker")
    response = client.invoke_endpoint(
        EndpointName=endpoint_name, ContentType=content_type, Body=encoded_json
    )
    return response

def transform_output(output: bytes) -> str:
    response_json = json.loads(output.read().decode("utf-8"))
    # return response_json
    return response_json[0][0]

In [ ]:
sentences2_1 = "타기관OTP 이용등록방법 알려주세요"
sentences2_2 = "다른곳OTP 사용방법 알려줘"
# sentences2_2 = "인터넷뱅킹 계정을 잃어버린 경우 어떻게 하는지 알려주세요"

payload_2_1 = {
    "inputs" : sentences2_1
}

payload_2_2 = {
    "inputs" : sentences2_2
}

# 첫번째 문장
query_response = query_endpoint_embedding_with_json_payload(
    json.dumps(payload_2_1).encode("utf-8"), endpoint_name=endpoint_name
)

emb_1 = transform_output(query_response['Body'])
print("첫문장 임베딩 사이즈: ", len(emb_1))

# 두번째 문장
query_response = query_endpoint_embedding_with_json_payload(
    json.dumps(payload_2_2).encode("utf-8"), endpoint_name=endpoint_name
)
 
emb_2 = transform_output(query_response['Body'])
print("두번째 문장 임베딩 사이즈: ", len(emb_2))

In [ ]:
def show_embedding_score3(emb1, emb2):

    embeddings_0 = torch.Tensor(emb1) 
    embeddings_1 = torch.Tensor(emb2)

    score01 = cal_score(embeddings_0, embeddings_1)

    print(score01)

show_embedding_score3(emb_1, emb_2) 

In [ ]:
# 나중에 endpoint name을 활용할 예정이라면 해당 값을 저장합니다.
embedding_endpoint_name = endpoint_name
%store embedding_endpoint_name

## 5. Delete endpoint

- 위에서 사용한 SageMaker endpoint는 EC2 인스턴스를 할당받아서 동작하기 때문에 사용하지 않을 때는 삭제해야 비용이 발생하지 않습니다.

In [ ]:
class clean_up():
    
    def __init__(self, ):    
        pass
    
    def delete_endpoint(self, client, endpoint_name ,is_del_model=True):
        
        response = client.describe_endpoint(EndpointName=endpoint_name)
        EndpointConfigName = response['EndpointConfigName']

        response = client.describe_endpoint_config(EndpointConfigName=EndpointConfigName)
        model_name = response['ProductionVariants'][0]['ModelName']    

        if is_del_model: # 모델도 삭제 여부 임.
            client.delete_model(ModelName=model_name)    

        client.delete_endpoint(EndpointName=endpoint_name)
        client.delete_endpoint_config(EndpointConfigName=EndpointConfigName)    

        print(f'--- Deleted model: {model_name}')
        print(f'--- Deleted endpoint: {endpoint_name}')
        print(f'--- Deleted endpoint_config: {EndpointConfigName}')  

In [ ]:
# clean = clean_up()
# sm_client = boto3.client('sagemaker')

# clean.delete_endpoint(sm_client, endpoint_name ,is_del_model=True)